In [17]:
import os
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from sympy import false

colors=px.colors.qualitative.D3

In [18]:
ram = [1, 2, 4, 8, 12, 16]
methods = ['Vanilla', 'XBZRLE', 'Compress', 'Dtrack', 'HBFDP_SHA1', 'HBFDP_MD5', 'HBFDP_Murmur3', 'XBZRLE_HBFDP_SHA1',
           'XBZRLE_HBFDP_MD5', 'XBZRLE_HBFDP_Murmur3', 'Compress_HBFDP_SHA1', 'Compress_HBFDP_MD5',
           'Compress_HBFDP_Murmur3']
#
# method_colors = {'Vanilla': colors[0], 'XBZRLE': colors[1], 'Compress': colors[2], 'HBFDP_SHA1': colors[3], 'HBFDP_MD5': colors[4], 'HBFDP_Murmur3': colors[5], 'Compress_HBFDP_SHA1': colors[3], 'Compress_HBFDP_MD5': colors[4], 'Compress_HBFDP_Murmur3': colors[5], 'XBZRLE_HBFDP_SHA1': colors[3], 'XBZRLE_HBFDP_MD5': colors[4], 'XBZRLE_HBFDP_Murmur3': colors[5] }
# markers = {'Vanilla': '', 'XBZRLE': '\\', 'Compress': 'x', 'HBFDP_SHA1': '/', 'HBFDP_MD5': '\\', 'HBFDP_Murmur3': '+', 'Compress_HBFDP_SHA1': '/', 'Compress_HBFDP_MD5': '\\', 'Compress_HBFDP_Murmur3': '+', 'XBZRLE_HBFDP_SHA1': '/', 'XBZRLE_HBFDP_MD5': '\\', 'XBZRLE_HBFDP_Murmur3': '+'}

method_colors = {'Vanilla': colors[0], 'XBZRLE': colors[1], 'Compress': colors[2], 'Dtrack': colors[3], 'Compress_Dtrack': colors[4], 'XBZRLE_Dtrack': colors[5], 'HBFDP_SHA1': colors[6], 'HBFDP_MD5': colors[7], 'HBFDP_Murmur3': colors[9], }
markers = {'Vanilla': '', 'XBZRLE': '\\', 'Compress': 'x', 'HBFDP_SHA1': '|', 'HBFDP_MD5': '.', 'HBFDP_Murmur3': '-', 'Dtrack': '/', 'Compress_Dtrack': '\\', 'XBZRLE_Dtrack': '+'}

def lighten_color(hex_color, percentage):
    """Lightens a given hex color by blending it with white."""
    # Ensure the input is valid
    hex_color = hex_color.lstrip('#')
    if len(hex_color) != 6:
        raise ValueError("Hex color must be in the format #RRGGBB")

    # Convert hex to RGB
    r, g, b = int(hex_color[0:2], 16), int(hex_color[2:4], 16), int(hex_color[4:6], 16)

    # Lighten by blending with white
    r = int(r + (255 - r) * (percentage / 100))
    g = int(g + (255 - g) * (percentage / 100))
    b = int(b + (255 - b) * (percentage / 100))

    # Convert back to hex
    return f"#{r:02X}{g:02X}{b:02X}"

# marker_colors={'Vanilla': lighten_color(colors[9], 80), 'XBZRLE': lighten_color(colors[7], 80), 'Compress': lighten_color(colors[0], 80),'HBFDP_SHA1': lighten_color(colors[1], 80), 'HBFDP_MD5': lighten_color(colors[2], 80), 'HBFDP_Murmur3': lighten_color(colors[3], 80),'Compress_HBFDP_SHA1': lighten_color(colors[1], 80), 'Compress_HBFDP_MD5': lighten_color(colors[2], 80), 'Compress_HBFDP_Murmur3': lighten_color(colors[3], 80),'XBZRLE_HBFDP_SHA1': lighten_color(colors[1], 80), 'XBZRLE_HBFDP_MD5': lighten_color(colors[2], 80), 'XBZRLE_HBFDP_Murmur3': lighten_color(colors[3], 80)}

marker_colors={'Vanilla': lighten_color(colors[9], 80), 'XBZRLE': lighten_color(colors[7], 80), 'Compress': lighten_color(colors[0], 80), 'HBFDP_SHA1': lighten_color(colors[4], 80), 'HBFDP_MD5': lighten_color(colors[5], 80), 'HBFDP_Murmur3': lighten_color(colors[6], 80), 'Dtrack': lighten_color(colors[1], 80), 'Compress_Dtrack': lighten_color(colors[2], 80), 'XBZRLE_Dtrack': lighten_color(colors[3], 80)}

In [19]:
def mig_time_plot(plot_column="Total-Time", y_title="Total Migration Time (s)", workload="Sysbench",
                  mig_methods=methods, tag="", ram_sizes=ram, error=True):
    mig_data = {}
    for mig_method in mig_methods:
        mig_data[mig_method] = {"data": [], "deviation": []}
        for ram_size in ram_sizes:
            ram_size_mb = str(ram_size * 1024)
            if os.path.exists(f'./data/{mig_method}/{workload}/{ram_size_mb}.csv'):
                data = pd.read_csv(f'./data/{mig_method}/{workload}/{ram_size_mb}.csv')
            # if os.path.exists(f'./HBFDP/{mig_method}/{workload}/{ram_size_mb}.csv'):
            #     data = pd.read_csv(f'./HBFDP/{mig_method}/{workload}/{ram_size_mb}.csv')
                filtered_data = data[data['Outlier'] != 'Yes']
                if plot_column == "Total-Time" or plot_column == "Downtime":
                    mig_data[mig_method]["data"].append(filtered_data[plot_column].mean() / 1000)
                    mig_data[mig_method]["deviation"].append(filtered_data[plot_column].std() / 1000)
                else:
                    mig_data[mig_method]["data"].append(filtered_data[plot_column].mean())
                    mig_data[mig_method]["deviation"].append(filtered_data[plot_column].std())

    data = pd.DataFrame({'RAM Size (GB)': [str(ram_size) for ram_size in ram_sizes]})

    count = 1
    for mig_method in mig_methods:
        if len(mig_data[mig_method]["data"]) <= 6:
            data.insert(column=mig_method, value=mig_data[mig_method]["data"], loc=count)
            count += 1

    layout = go.Layout(
        xaxis=dict(title='RAM Size (GB)'),
        yaxis=dict(title=y_title),
        barmode='group',
        template='simple_white'
    )

    trace = []

    for mig_method in mig_methods:
        name = ""
        for word in mig_method.split("_"):
            name = f'{name} {word}'

        if "Vanilla" in name:
            name = " Vanilla Pre-Copy"

        if len(mig_data[mig_method]["data"]) <= 6:
            if error:
                trace.append(go.Bar(x=data['RAM Size (GB)'], y=data[mig_method], name=f'{name}',
                                    error_y=dict(type='data', array=mig_data[mig_method]["deviation"]),
                                    marker_color=method_colors[mig_method],
                                    marker_pattern_shape=markers[mig_method],
                                    marker_pattern_fgcolor=marker_colors[mig_method],
                                    ))
            else:
                trace.append(go.Bar(x=data['RAM Size (GB)'], y=data[mig_method], name=f'{name}',
                                    marker_color=method_colors[mig_method],
                                    marker_pattern_shape=markers[mig_method],
                                    marker_pattern_fgcolor=marker_colors[mig_method],
                                    ))

    fig = go.Figure(data=trace, layout=layout)
    fig.update_traces(
        marker_pattern=dict(
            size=6,
            solidity=0.2,
            # fgopacity=0.8
        ),
        selector=dict(type='bar')
    )

    wx = 1.2
    if 'XBZRLE_HBFDP_Murmur3' in mig_methods or 'Compress_HBFDP_Murmur3' in mig_methods:
        wx = 1.35

    fig.update_layout(
        width=1200,  # 0.45 text width
        height=600,
        font=dict(family="Arial", size=24, color="black"),
        margin=dict(t=15, r=10, l=70, b=45, autoexpand=True),
        yaxis_title_standoff=10,
        xaxis_title_standoff=8,
        # legend=dict(
        #     orientation="v",  # Vertical orientation for legend items
        #     yanchor="middle",  # Anchor the legend vertically to the middle
        #     y=0.5,  # Place the legend at the vertical center of the plot
        #     xanchor="right",  # Anchor the legend to the right side
        #     x=wx,  # Position the legend just outside the plot area on the right
        # ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.25,
            xanchor="center",
            x=0.5,
            font=dict(size=18)
        ),
        xaxis_title=dict(
            text=fig.layout.xaxis.title.text,
            font=dict(size=20)  # Adjust x-axis title font size
        ),
        yaxis_title=dict(
            text=fig.layout.yaxis.title.text,
            font=dict(size=20)  # Adjust y-axis title font size
        ),
        paper_bgcolor='#F9F9F9',  # Color of the entire figure
        plot_bgcolor='#F9F9F9'
    )
    fig.write_image(f"./RAM-Size-vs-{plot_column}.{workload}.{tag}.png", scale=4, width=1200, height=600)
    # /plots/new
    # fig.write_image(f"./plots/hbfdp/RAM-Size-vs-{plot_column}.{workload}.{tag}.png", scale=4, width=1200, height=600)

    fig.show()

# TMT vs RAM Size - Memcached
```Vanilla, XBZRLE, Compress, HBFDP-SHA1, HBFDP-MD5, HBFDP-Murmur3```

In [20]:
# mig_time_plot("Total-Time", "Total Migration Time (s)", "Memcached",
#               # ['Vanilla', 'XBZRLE', 'Compress', 'HBFDP_SHA1', 'HBFDP_MD5', 'HBFDP_Murmur3'], "V-X-C-HS1-HM5-HM3")

# TMT vs RAM Size - Memcached
```Vanilla, XBZRLE, Compress, HBFDP-SHA1, HBFDP-MD5, HBFDP-Murmur3```

In [21]:
# mig_time_plot("Total-Time", "Total Migration Time (s)", "Memcached",
#               ['Vanilla', 'XBZRLE', 'Compress', 'Compress_HBFDP_SHA1', 'Compress_HBFDP_MD5',
#            'Compress_HBFDP_Murmur3'], "V-X-C-CHS1-CHM5-CHM3")

# TMT vs RAM Size - Memcached
```Vanilla, XBZRLE, Compress, HBFDP-SHA1, HBFDP-MD5, HBFDP-Murmur3```

In [22]:
# mig_time_plot("Total-Time", "Total Migration Time (s)", "Memcached",
#               ['Vanilla', 'XBZRLE', 'Compress', 'XBZRLE_HBFDP_SHA1',
#            'XBZRLE_HBFDP_MD5', 'XBZRLE_HBFDP_Murmur3'], "V-X-C-XHS1-XHM5-XHM3")

# TMT vs RAM Size - Memcached
```Vanilla, XBZRLE, Compress, Dtrack, HBFDP-SHA1, HBFDP-MD5, HBFDP-Murmur3```

In [23]:
mig_time_plot("Total-Time", "Total Migration Time (s)", "Memcached",
              ['Vanilla', 'XBZRLE', 'Compress', 'HBFDP_SHA1'], "V-X-C-HS1")

# TMT vs RAM Size - Memcached
```Compress, Dtrack, Compress_Dtrack, XBZRLE_Dtrack```

In [24]:
mig_time_plot("Total-Time", "Total Migration Time (s)", "Memcached",
              ['Vanilla', 'XBZRLE', 'Compress', 'HBFDP_SHA1', 'Dtrack', 'Compress_Dtrack', 'XBZRLE_Dtrack'], "V-X-C-HS1-D-CD-XD")

# DT vs RAM Size - Memcached
```Compress, Dtrack, Compress_Dtrack, XBZRLE_Dtrack```

In [25]:
# mig_time_plot("Downtime", "Downtime (s)", "Memcached",
#               ['Vanilla', 'XBZRLE', 'Compress', 'HBFDP_SHA1', 'Dtrack', 'Compress_Dtrack', 'XBZRLE_Dtrack'], "V-X-C-HS1-D-CD-XD", error=False)

# TMT vs RAM Size - Sysbench
```Vanilla, XBZRLE, Compress, Dtrack, HBFDP-SHA1, HBFDP-MD5, HBFDP-Murmur3```

In [26]:
# mig_time_plot("Total-Time", "Total Migration Time (s)", "Sysbench",
#               ['Vanilla', 'XBZRLE', 'Compress', 'HBFDP_SHA1', 'HBFDP_MD5', 'HBFDP_Murmur3', 'Dtrack'], "V-X-C-HS1-HM5-HM3-D")

# TMT vs RAM Size - Sysbench
```Compress, Dtrack, Compress_Dtrack, XBZRLE_Dtrack```


In [27]:
# mig_time_plot("Total-Time", "Total Migration Time (s)", "Sysbench",
#               ['Vanilla', 'XBZRLE', 'Compress', 'Dtrack', 'Compress_Dtrack', 'XBZRLE_Dtrack'], "V-X-C-D-CD-XD")

# TMT vs RAM Size - OLTP
```Vanilla, XBZRLE, Compress, Dtrack, HBFDP-SHA1, HBFDP-MD5, HBFDP-Murmur3```

In [28]:
# mig_time_plot("Total-Time", "Total Migration Time (s)", "Oltp",
#               ['Vanilla', 'XBZRLE', 'Compress', 'HBFDP_SHA1', 'HBFDP_MD5', 'HBFDP_Murmur3', 'Dtrack'], "V-X-C-HS1-HM5-HM3-D")

# TMT vs RAM Size - OLTP
```Compress, Dtrack, Compress_Dtrack, XBZRLE_Dtrack```


In [29]:
# mig_time_plot("Total-Time", "Total Migration Time (s)", "Oltp",
#               ['Vanilla', 'XBZRLE', 'Compress', 'Dtrack', 'Compress_Dtrack', 'XBZRLE_Dtrack'], "V-X-C-D-CD-XD")

# TMT vs RAM Size - Memcached
```Compress, Compress_HBFDP_SHA1, Compress_HBFDP_MD5, Compress_HBFDP_Murmur3```

In [30]:
# mig_time_plot("Total-Time", "Total Migration Time (s)", "Memcached",
#               ['Compress', 'Compress_HBFDP_SHA1', 'Compress_HBFDP_MD5', 'Compress_HBFDP_Murmur3'], "C-CHS1-CHM5-CHM3")

# TMT vs RAM Size - Memcached
```XBZRLE, XBZRLE_HBFDP_SHA1, XBZRLE_HBFDP_MD5, XBZRLE_HBFDP_Murmur3```

In [31]:
# mig_time_plot("Total-Time", "Total Migration Time (s)", "Memcached",
#               ['XBZRLE', 'XBZRLE_HBFDP_SHA1', 'XBZRLE_HBFDP_MD5', 'XBZRLE_HBFDP_Murmur3'], "X-XHS1-XHM5-XHM3")

# Downtime vs RAM Size - Memcached

In [32]:
mig_time_plot("Downtime", "Downtime (s)", "Memcached",
              ['Vanilla', 'XBZRLE', 'Compress', 'HBFDP_SHA1', 'Dtrack', 'Compress_Dtrack', 'XBZRLE_Dtrack'], "V-X-C-HS1-D-CD-XD", error=false)